In [ ]:
import simpy 
import random
import numpy as np
import pandas as pd
from scipy.stats import norm

# Simulation parameters
RANDOM_SEED = 42
NUM_STAFF = 5        # Reduced number of staff to create variability
SIM_TIME = 2880         # Simulation time in minutes (48 hours)
ARRIVAL_RATE = 20       # Patients per hour (increased rate to create queues)
SERVICE_TIME_MEAN = 20  # Mean service time in minutes
SERVICE_TIME_STD = 5    # Variability in service time
NUM_SIMULATIONS = 100   # Number of simulation runs

# Patient arrival process
def patient(env, name, staff, wait_times):
    arrival_time = env.now

    # Request a staff member
    with staff.request() as request:
        yield request

        # Randomize minimum wait time
        min_wait_time = random.randint(5, 20)  # Random wait between 5 and 20 minutes
        min_wait_end = max(arrival_time + min_wait_time, env.now)
        yield env.timeout(max(0, min_wait_end - env.now))

        # Record actual wait time
        wait_time = env.now - arrival_time
        wait_times.append(wait_time)

        # Add variability to service time
        service_time = max(0, random.normalvariate(SERVICE_TIME_MEAN, SERVICE_TIME_STD))
        yield env.timeout(service_time)

# Generate patients
def patient_generator(env, staff, wait_times):
    i = 0
    while True:
        # Interarrival time based on ARRIVAL_RATE
        interarrival_time = random.expovariate(ARRIVAL_RATE / 60.0)
        yield env.timeout(interarrival_time)
        i += 1
        env.process(patient(env, f"Patient {i}", staff, wait_times))

# Function to run a single simulation
random.seed(RANDOM_SEED)
def run_simulation():
    env = simpy.Environment()
    staff = simpy.Resource(env, capacity=NUM_STAFF)
    wait_times = []

    env.process(patient_generator(env, staff, wait_times))
    env.run(until=SIM_TIME)

    # Handle case where no wait times are recorded
    if len(wait_times) == 0:
        return {
            "average_wait_time": 0,
            "min_wait_time": 0,
            "max_wait_time": 0,
            "patients_served": 0
        }

    # Return results
    return {
        "average_wait_time": np.mean(wait_times),
        "min_wait_time": np.min(wait_times),
        "max_wait_time": np.max(wait_times),
        "patients_served": len(wait_times)
    }

# Run multiple simulations
all_results = []
for sim in range(NUM_SIMULATIONS):
    result = run_simulation()
    all_results.append(result)

# Convert results to a DataFrame
results_df = pd.DataFrame(all_results)

# Calculate confidence intervals for average wait time
mean_avg_wait = results_df["average_wait_time"].mean()
std_avg_wait = results_df["average_wait_time"].std()

# Handle zero variance or invalid values
if std_avg_wait == 0 or np.isnan(mean_avg_wait) or np.isnan(std_avg_wait):
    confidence_interval = (mean_avg_wait, mean_avg_wait)  # Single point CI
else:
    confidence_interval = norm.interval(0.95, loc=mean_avg_wait, scale=std_avg_wait / np.sqrt(NUM_SIMULATIONS))

# Report results
print("\nSimulation Results Across Multiple Runs:")
print(f"Mean Average Wait Time: {mean_avg_wait:.2f} minutes")
if confidence_interval[0] is not None and confidence_interval[1] is not None:
    print(f"95% Confidence Interval for Average Wait Time: {confidence_interval[0]:.2f} to {confidence_interval[1]:.2f} minutes")
else:
    print("95% Confidence Interval could not be calculated.")
print(f"Mean Patients Served: {results_df['patients_served'].mean():.0f}")


